# 

In [1]:
VER=1

import numpy as np, pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

In [2]:


fertilizer_name_dict = {
    '10-26-26': 0, '14-35-14': 1, '17-17-17': 2, '20-20': 3,
    '28-28': 4, 'DAP': 5, 'Urea': 6
}
inverse_fert_dict = {v: k for k, v in fertilizer_name_dict.items()}


train = pd.read_csv("/kaggle/input/playground-series-s5e6/train.csv")
true = train["Fertilizer Name"].map(fertilizer_name_dict).values
train.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,1,27,69,65,Sandy,Millets,30,6,18,28-28
2,2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,4,35,58,43,Red,Paddy,37,2,16,DAP


In [3]:
true

array([4, 4, 2, ..., 0, 3, 6])

In [4]:
import os
import numpy as np
import pandas as pd

print("Loading files from multiple paths...")

# Birden fazla path tanımla
PATHS = [
    '/kaggle/input/denemeeee/',
    '/kaggle/input/iste-buuuuuuu/',
    '/kaggle/input/lgbm-gosss/',
    '/kaggle/input/topss/'
    
]

# Her path için ayrı dosya listesi tutacağız
all_oof_files = []
all_sub_files = []
file_path_mapping = {}  # Dosya adı -> path mapping

for path in PATHS:
    print(f"\nChecking path: {path}")
    if os.path.exists(path):
        file_list = os.listdir(path)
        
        # Bu path'deki .npy dosyalarını filtrele
        path_oof_files = [f for f in file_list if f.endswith(".npy") and "oof" in f]
        path_sub_files = [f for f in file_list if f.endswith(".npy") and "submission" in f]
        
        print(f"  Found {len(path_oof_files)} OOF files and {len(path_sub_files)} submission files")
        
        # Dosyaları toplam listeye ekle
        all_oof_files.extend(path_oof_files)
        all_sub_files.extend(path_sub_files)
        
        # Her dosya için path mapping'i kaydet
        for f in path_oof_files + path_sub_files:
            file_path_mapping[f] = path
            
        print(f"  OOF files: {path_oof_files}")
        print(f"  SUB files: {path_sub_files}")
    else:
        print(f"  Path does not exist: {path}")

print(f"\nTotal found: {len(all_oof_files)} OOF files and {len(all_sub_files)} submission files")

def get_model_key(fname):
    """Dosya adından model anahtarını çıkarır"""
    try:
        # CatBoost trial dosyaları
        if "cat_oof_trial_" in fname:
            trial_num = fname.replace("cat_oof_trial_", "").replace(".npy", "")
            return f"cat_trial_{trial_num}"
        elif "cat_submission_trial_" in fname:
            trial_num = fname.replace("cat_submission_trial_", "").replace(".npy", "")
            return f"cat_trial_{trial_num}"
        
        # Top4 dosyaları - DÜZELTİLDİ
        elif "top4_1_oof.npy" in fname:
            return "top4_1"
        elif "top4_1_submission.npy" in fname:
            return "top4_1"
        elif "top4_2_oof.npy" in fname:
            return "top4_2"
        elif "top4_2_submission.npy" in fname:
            return "top4_2"
        elif "top4_3_oof.npy" in fname:
            return "top4_3"
        elif "top4_3_submission.npy" in fname:
            return "top4_3"
        elif "top4_4_oof.npy" in fname:
            return "top4_4"
        elif "top4_4_submission.npy" in fname:
            return "top4_4"
        
        # LGBM dosyaları
        elif "lgbm_goss_oof_trial_" in fname:
            trial_num = fname.replace("lgbm_goss_oof_trial_", "").replace(".npy", "")
            return f"lgbm_goss_trial_{trial_num}"
        elif "lgbm_goss_submission_trial_" in fname:
            trial_num = fname.replace("lgbm_goss_submission_trial_", "").replace(".npy", "")
            return f"lgbm_goss_trial_{trial_num}"
        
        # XGBoost dosyaları - YENİ PATTERN EKLENDİ
        elif "xgb_1.4m_train_oof_trial_" in fname:
            trial_num = fname.replace("xgb_1.4m_train_oof_trial_", "").replace(".npy", "")
            return f"xgb_1.4m_train_trial_{trial_num}"
        elif "xgb_1.4m_train_submission_trial_" in fname:
            trial_num = fname.replace("xgb_1.4m_train_submission_trial_", "").replace(".npy", "")
            return f"xgb_1.4m_train_trial_{trial_num}"
        # LGBM_goss 1.4m train dosyaları - DÜZELTME
        elif "lgbm_goss_1.4m_train_oof_trial_" in fname:
            trial_num = fname.replace("lgbm_goss_1.4m_train_oof_trial_", "").replace(".npy", "")
            return f"lgbm_goss_1.4m_train_trial_{trial_num}"
        elif "lgbm_goss_1.4m_train_submission_trial_" in fname:
            trial_num = fname.replace("lgbm_goss_1.4m_train_submission_trial_", "").replace(".npy", "")
            return f"lgbm_goss_1.4m_train_trial_{trial_num}"
        
        # Eski XGBoost dosyaları (varsa)
        elif "xgb_oof_trial_" in fname:
            trial_num = fname.replace("xgb_oof_trial_", "").replace(".npy", "")
            return f"xgb_trial_{trial_num}"
        elif "xgb_submission_trial_" in fname:
            trial_num = fname.replace("xgb_submission_trial_", "").replace(".npy", "")
            return f"xgb_trial_{trial_num}"
        elif "XGBoost_best_oof.npy" in fname:
            return "XGBoost_best"
        elif "XGBoost_best_submission.npy" in fname:
            return "XGBoost_best"
        
        # MLP seed dosyaları
        elif "MLP_seed_" in fname and "_oof.npy" in fname:
            return fname.replace("_oof.npy", "")
        elif "MLP_seed_" in fname and "_submission.npy" in fname:
            return fname.replace("_submission.npy", "")
        
        # TabNET dosyaları
        elif "TabNET_cv_" in fname and "_oof.npy" in fname:
            return fname.replace("_oof.npy", "")
        elif "TabNET_cv_" in fname and "_submission.npy" in fname:
            return fname.replace("_submission.npy", "")
        
        # Genel OOF ve submission dosyaları
        elif "_oof.npy" in fname:
            return fname.replace("_oof.npy", "")
        elif "_submission.npy" in fname:
            return fname.replace("_submission.npy", "")
        
        return None
    except:
        return None
# Eşleştirme için dictionary oluştur
oof_dict = {}
sub_dict = {}

print("\nProcessing OOF files:")
for f in sorted(all_oof_files):
    model_key = get_model_key(f)
    if model_key:
        oof_dict[model_key] = f
        print(f"  {f} -> {model_key} (from {file_path_mapping[f]})")
    else:
        print(f"  {f} -> NOT MATCHED (from {file_path_mapping[f]})")

print("\nProcessing submission files:")
for f in sorted(all_sub_files):
    model_key = get_model_key(f)
    if model_key:
        sub_dict[model_key] = f
        print(f"  {f} -> {model_key} (from {file_path_mapping[f]})")
    else:
        print(f"  {f} -> NOT MATCHED (from {file_path_mapping[f]})")

# Eşleşen dosyaları bul
matched_keys = set(oof_dict.keys()) & set(sub_dict.keys())
unmatched_oof = set(oof_dict.keys()) - set(sub_dict.keys())
unmatched_sub = set(sub_dict.keys()) - set(oof_dict.keys())

print(f"\n=== MATCHING RESULTS ===")
print(f"Matched pairs: {len(matched_keys)}")
print(f"Unmatched OOF files: {len(unmatched_oof)}")
print(f"Unmatched submission files: {len(unmatched_sub)}")

if unmatched_oof:
    print("\nUnmatched OOF files:")
    for key in sorted(unmatched_oof):
        print(f"  {key}: {oof_dict[key]} (from {file_path_mapping[oof_dict[key]]})")

if unmatched_sub:
    print("\nUnmatched submission files:")
    for key in sorted(unmatched_sub):
        print(f"  {key}: {sub_dict[key]} (from {file_path_mapping[sub_dict[key]]})")

print(f"\nFound {len(matched_keys)} matching pairs:")
for key in sorted(matched_keys):
    oof_file = oof_dict[key]
    sub_file = sub_dict[key]
    print(f"  {key}:")
    print(f"    OOF: {oof_file} (from {file_path_mapping[oof_file]})")
    print(f"    SUB: {sub_file} (from {file_path_mapping[sub_file]})")

# Sırala
matched_keys = sorted(matched_keys)

# Dosyaları yükle
x_train, x_test, files, file_paths = [], [], [], []

print(f"\n=== LOADING DATA ===")
print(f"Loading {len(matched_keys)} model pairs...")

for i, model_key in enumerate(matched_keys):
    oof_file = oof_dict[model_key]
    sub_file = sub_dict[model_key]
    oof_path = file_path_mapping[oof_file]
    sub_path = file_path_mapping[sub_file]
    
    print(f"\n[{i+1}/{len(matched_keys)}] Loading: {model_key}")
    print(f"  OOF: {oof_file} (from {oof_path})")
    print(f"  SUB: {sub_file} (from {sub_path})")
    
    try:
        oof_data = np.load(os.path.join(oof_path, oof_file))
        sub_data = np.load(os.path.join(sub_path, sub_file))
        
        x_train.append(oof_data)
        x_test.append(sub_data)
        files.append(oof_file)
        file_paths.append(oof_path)
        
        print(f"  ✓ Loaded - OOF: {oof_data.shape}, SUB: {sub_data.shape}")
        
        # Temel istatistikler
        print(f"  OOF range: [{oof_data.min():.6f}, {oof_data.max():.6f}]")
        print(f"  SUB range: [{sub_data.min():.6f}, {sub_data.max():.6f}]")
        
    except Exception as e:
        print(f"  ✗ Error loading: {e}")

print(f"\n=== FINAL SUMMARY ===")
print(f"Successfully loaded: {len(x_train)} model pairs")
if x_train:
    print(f"OOF shapes: {[arr.shape for arr in x_train]}")
    print(f"SUB shapes: {[arr.shape for arr in x_test]}")
    print(f"Model names: {files}")
    
    # Ensemble için hazırlık
    print(f"\n=== ENSEMBLE PREPARATION ===")
    print(f"x_train = np.column_stack(x_train)  # Shape: {x_train[0].shape[0]} x {len(x_train)}")
    print(f"x_test = np.column_stack(x_test)    # Shape: {x_test[0].shape[0]} x {len(x_test)}")
    
    # Model türlerini say
    model_types = {}
    path_distribution = {}
    
    for i, name in enumerate(files):
        # Dosya adından model tipini çıkar
        if 'cat_' in name:
            model_type = 'cat'
        elif 'lgbm_' in name:
            model_type = 'lgbm'
        elif 'MLP_' in name:
            model_type = 'MLP'
        elif 'TabNET_' in name:
            model_type = 'TabNET'
        elif 'xgb_' in name or 'XGBoost_' in name:
            model_type = 'xgb'
        else:
            model_type = 'other'
        model_types[model_type] = model_types.get(model_type, 0) + 1
        
        # Path dağılımını say
        path = file_paths[i]
        path_distribution[path] = path_distribution.get(path, 0) + 1
    
    print(f"\nModel distribution:")
    for model_type, count in sorted(model_types.items()):
        print(f"  {model_type}: {count} models")
    
    print(f"\nPath distribution:")
    for path, count in sorted(path_distribution.items()):
        print(f"  {path}: {count} models")

    # Actual ensemble arrays oluştur
    x_train_ensemble = np.column_stack(x_train)
    x_test_ensemble = np.column_stack(x_test)
    
    print(f"\n=== ENSEMBLE ARRAYS CREATED ===")
    print(f"x_train_ensemble shape: {x_train_ensemble.shape}")
    print(f"x_test_ensemble shape: {x_test_ensemble.shape}")

# Dosya listesini DataFrame olarak kaydet
if matched_keys:
    df_files = pd.DataFrame({
        'model_key': matched_keys,
        'file_name': files,
        'file_path': file_paths,
        'oof_file': [oof_dict[key] for key in matched_keys],
        'sub_file': [sub_dict[key] for key in matched_keys],
        'oof_path': [file_path_mapping[oof_dict[key]] for key in matched_keys],
        'sub_path': [file_path_mapping[sub_dict[key]] for key in matched_keys]
    })
    print(f"\nFile mapping DataFrame created with {len(df_files)} rows")
    print(df_files.head())
else:
    print("\nNo matched pairs found. Check your file naming patterns.")
    print("Available files by path:")
    for path in PATHS:
        if os.path.exists(path):
            print(f"\n{path}:")
            oof_files_in_path = [f for f in all_oof_files if file_path_mapping[f] == path]
            sub_files_in_path = [f for f in all_sub_files if file_path_mapping[f] == path]
            print(f"  OOF files: {oof_files_in_path}")
            print(f"  SUB files: {sub_files_in_path}")

Loading files from multiple paths...

Checking path: /kaggle/input/denemeeee/
  Found 14 OOF files and 14 submission files
  OOF files: ['xgb_1.4m_train_oof_trial_13.npy', 'xgb_1.4m_train_oof_trial_1.npy', 'xgb_1.4m_train_oof_trial_8.npy', 'xgb_1.4m_train_oof_trial_2.npy', 'xgb_1.4m_train_oof_trial_0.npy', 'xgb_1.4m_train_oof_trial_6.npy', 'xgb_1.4m_train_oof_trial_11.npy', 'xgb_1.4m_train_oof_trial_4.npy', 'xgb_1.4m_train_oof_trial_3.npy', 'xgb_1.4m_train_oof_trial_7.npy', 'xgb_1.4m_train_oof_trial_12.npy', 'xgb_1.4m_train_oof_trial_5.npy', 'xgb_1.4m_train_oof_trial_10.npy', 'xgb_1.4m_train_oof_trial_9.npy']
  SUB files: ['xgb_1.4m_train_submission_trial_12.npy', 'xgb_1.4m_train_submission_trial_10.npy', 'xgb_1.4m_train_submission_trial_6.npy', 'xgb_1.4m_train_submission_trial_2.npy', 'xgb_1.4m_train_submission_trial_9.npy', 'xgb_1.4m_train_submission_trial_8.npy', 'xgb_1.4m_train_submission_trial_1.npy', 'xgb_1.4m_train_submission_trial_0.npy', 'xgb_1.4m_train_submission_trial_11.npy

In [5]:
x_train = np.stack(x_train).T
print("Our combined OOF have shape:",x_train.shape)

x_test = np.stack(x_test).T
print("Our combined PRED have shape:",x_test.shape)

Our combined OOF have shape: (7, 750000, 59)
Our combined PRED have shape: (7, 250000, 59)


In [6]:
x_train = np.stack(x_train, axis=-1).transpose(0, 2, 1)  # shape: (750000, 7, n_models)
x_test  = np.stack(x_test, axis=-1).transpose(0, 2, 1)

In [7]:
print(x_train.shape)  # shape kontrolü
print(x_train[:, :, 0][:5])  # ilk 5 sample, model-0 tahminleri
print(np.sum(x_train[:, :, 0], axis=1)[:5])  # her sample için toplam olasılık
print(np.max(x_train[:, :, 0], axis=1)[:5])  # en yüksek olasılıklar


(750000, 7, 59)
[[0.14941223 0.1433233  0.14189668 0.14585286 0.1632528  0.1289604
  0.12730168]
 [0.13381448 0.12455395 0.14200827 0.16198124 0.16419832 0.14349176
  0.12995201]
 [0.14644124 0.15534693 0.15847446 0.13851431 0.14364673 0.12346108
  0.13411519]
 [0.13682742 0.14627595 0.1452865  0.14172171 0.14998427 0.13687257
  0.14303151]
 [0.14923657 0.14739461 0.16275784 0.14716931 0.15181448 0.12802447
  0.11360271]]
[0.99999996 1.00000004 0.99999993 0.99999994 1.        ]
[0.1632528  0.16419832 0.15847446 0.14998427 0.16275784]


In [8]:
def apk(actual, predicted, k=3):
    if len(predicted) > k:
        predicted = predicted[:k]
    score = num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=3):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

In [9]:
def multiple_mapk_scores(actual, predicted, k=3):
    """
    Compute MAP@k scores for multiple models using CuPy (based on apk/mapk logic).
    
    Parameters
    ----------
    actual : cupy.ndarray
        Shape = (N,) -- true class indices (single correct label per row)
    predicted : cupy.ndarray
        Shape = (N, C, K) -- predicted class probabilities

    Returns
    -------
    cupy.ndarray
        Shape = (K,) -- MAP@k scores per model
    """
    N, C, K = predicted.shape

    # Get top-k predicted class indices for each model
    topk_preds = cp.argsort(predicted, axis=1)[:, -k:, ::-1]  # shape: (N, k, K)

    # Broadcast actuals to shape (N, k, K)
    actual_exp = actual[:, None, None]

    # Compute hits: (N, k, K)
    hits = (topk_preds == actual_exp)

    # Cumulative precision: only first hit counts in apk-style (like apk())
    precision_at_k = hits / cp.arange(1, k + 1)[None, :, None]

    # Only the first match counts: apk style behavior
    apk_scores = cp.sum(precision_at_k, axis=1)  # shape: (N, K)

    mapk_scores = cp.mean(apk_scores, axis=0)  # shape: (K,)
    return mapk_scores


In [10]:
actual = [[t] for t in true]  # MAP@3 için list of list

def compute_metric_map3(p, true_labels):
    """
    p: shape = (n_samples, 7), her modelin OOF çıktısı
    true_labels: shape = (n_samples,), int class index'ler
    """
    top3 = np.argsort(p, axis=1)[:, -3:][:, ::-1]  # En yüksek 3 tahmin
    actual = [[t] for t in true_labels]
    predicted = top3.tolist()
    return mapk(actual, predicted, k=3)


best_score = -1
best_index = -1

for k, name in enumerate(files):
    score = compute_metric_map3(x_train[:, :, k], true)
    print(f'MAP@3 {score:.5f} {name}')
    
    if score > best_score:
        best_score = score
        best_index = k

print()
print(f'✅ Best single model is {files[best_index]} with MAP@3 = {best_score:.5f}')

MAP@3 0.29331 MLP_seed_2025_oof.npy
MAP@3 0.29422 MLP_seed_42_oof.npy
MAP@3 0.29432 MLP_seed_7_oof.npy
MAP@3 0.27919 TabNET.cv_0.279192_oof.npy
MAP@3 0.36263 XGBoost_best_oof.npy
MAP@3 0.32249 cat_oof_trial_0.npy
MAP@3 0.31970 cat_oof_trial_1.npy
MAP@3 0.32666 cat_oof_trial_10.npy
MAP@3 0.32930 cat_oof_trial_11.npy
MAP@3 0.32921 cat_oof_trial_12.npy
MAP@3 0.32910 cat_oof_trial_13.npy
MAP@3 0.31264 cat_oof_trial_14.npy
MAP@3 0.30619 cat_oof_trial_2.npy
MAP@3 0.31022 cat_oof_trial_3.npy
MAP@3 0.29594 cat_oof_trial_4.npy
MAP@3 0.32550 cat_oof_trial_5.npy
MAP@3 0.31506 cat_oof_trial_6.npy
MAP@3 0.30660 cat_oof_trial_7.npy
MAP@3 0.31937 cat_oof_trial_8.npy
MAP@3 0.30002 cat_oof_trial_9.npy
MAP@3 0.34880 lgbm_goss_1.4m_train_oof_trial_0.npy
MAP@3 0.34437 lgbm_goss_1.4m_train_oof_trial_1.npy
MAP@3 0.33248 lgbm_goss_1.4m_train_oof_trial_2.npy
MAP@3 0.34529 lgbm_goss_1.4m_train_oof_trial_3.npy
MAP@3 0.35627 lgbm_goss_1.4m_train_oof_trial_4.npy
MAP@3 0.33438 lgbm_goss_oof_trial_0.npy
MAP@3 0.337

In [11]:
import cupy as cp

def multiple_map3_scores(actual, predicted):
    """
    Compute MAP@3 scores for multiple models using GPU (CuPy) - FIXED VERSION
    
    Parameters:
    ----------
    actual : cupy.ndarray
        Shape = (N,)  -- true class indices
    predicted : cupy.ndarray
        Shape = (N, C, K) -- predicted probabilities for C classes and K models
    
    Returns:
    -------
    cupy.ndarray
        Shape = (K,) -- MAP@3 score for each model
    """
    N, C, K = predicted.shape
    
    # Top-3 class indices per sample, per model
    top3_preds = cp.argsort(predicted, axis=1, kind='stable')[:, -3:][:, ::-1]  # shape: (N, 3, K)
    
    # Expand true labels: (N,) → (N, 1, 1) → broadcasted to (N, 3, K)
    actual_exp = actual[:, None, None]  # shape = (N, 1, 1)
    
    # Compute hit matrix: whether true class is in top-3 (broadcasted match)
    hits = (top3_preds == actual_exp).astype(cp.float32)  # shape: (N, 3, K), float
    
    # ✅ FIX: Kümülatif hit sayısını hesapla (MAP@3'ün temel gereksinimi)
    cumulative_hits = cp.cumsum(hits, axis=1)  # shape: (N, 3, K)
    
    # ✅ FIX: Precision@k = cumulative_hits[k] / (k+1), sadece hit olan pozisyonlarda
    positions = cp.arange(1, 4)[None, :, None]  # [1, 2, 3] shape: (1, 3, 1)
    precision_at_k = hits * cumulative_hits / positions  # shape: (N, 3, K)
    
    # Her sample için AP score (Average Precision)
    ap_scores = cp.sum(precision_at_k, axis=1)  # shape: (N, K)
    
    # MAP@3 = ortalama AP score
    map3 = cp.mean(ap_scores, axis=0)  # shape: (K,)
    
    return map3

cv cok kaynak harciyor, oofler ve sublar zaten cv icerisinden geldigi icin buna gerek yok. 

In [12]:
import numpy as np
import cupy as cp
import gc

# --- SABİTLER ---
USE_NEGATIVE_WGT = False       # Negatif ağırlığa izin verilsin mi?
MAX_MODELS        = 1000      # Hill-climb döngüsünün en fazla kaç model ekleyeceği
TOL               = 1e-5      # Skor iyileşmesi için minimum eşik

# Ağırlık aralığını oluştur (GPU'da kullanmak üzere)
start = -0.50 if USE_NEGATIVE_WGT else 0.01
ww    = cp.arange(start, 0.51, 0.01)  # [-0.5, -0.49, …, 0.49, 0.5]

# x_train: np.array, shape=(n_samples, n_classes, n_models) - OOF predictions
# true:    np.array, shape=(n_samples,) - true labels
# x_test:  np.array, shape=(n_test, n_classes, n_models) - test predictions
# files:   list of length n_models, her modelin dosya/ad etiketi

print("🚀 Hill-Climbing Ensemble Başlıyor (CV'siz)...")

# GPU'ya taşı
x_train_gpu = cp.array(x_train)             
truth_gpu   = cp.array(true)             

# --- Başlangıç: En iyi tek model var mı diye kontrol et ---
print("🔍 En iyi başlangıç modeli aranıyor...")
init_scores = []
for k in range(x_train_gpu.shape[2]):
    preds_k = x_train_gpu[:, :, k][:, :, None]                     # (n, classes, 1)
    score_k = cp.max(multiple_map3_scores(truth_gpu, preds_k)).item()
    init_scores.append(score_k)

best_index = int(np.argmax(init_scores))
best_score = init_scores[best_index]
print(f"0️⃣ Başlangıç modeli \"{files[best_index]}\" MAP@3 = {best_score:.5f}")

# Ensemble başlangıcı
best_ensemble = x_train_gpu[:, :, best_index]   # (n_samples, n_classes)
models        = [best_index]
weights       = [1.0]
old_best      = best_score

# --- Hill-Climbing Döngüsü ---
print("\n⛰️ Hill-Climbing başlıyor...")
for step in range(1, MAX_MODELS+1):
    step_best_score = -np.inf
    step_best      = None  # (model_idx, weight, new_ensemble)
    
    # Henüz seçilmemiş her bir modeli dene
    for k in range(x_train_gpu.shape[2]):
        if k in models:
            continue
        candidate_model = x_train_gpu[:, :, k]  # (n_samples, n_classes)
        
        # Tüm ağırlık aralığını dener
        for w in ww:
            ens = best_ensemble * (1-w) + candidate_model * w
            sc  = cp.max(multiple_map3_scores(truth_gpu, ens[:, :, None])).item()
            if sc > step_best_score:
                step_best_score = sc
                step_best       = (k, float(w), ens)
    
    # Durma kriteri: yeterince küçük iyileşme
    if step_best_score - old_best < TOL:
        print(f"⏹️ İyileşme {step_best_score - old_best:.6f} < TOL ({TOL}) — hill-climb sonlandı")
        break
    
    # Adım güncellemesi
    k_sel, w_sel, ens_sel = step_best
    models.append(k_sel)
    weights.append(w_sel)
    best_ensemble = ens_sel
    old_best      = step_best_score
    print(f"{step:2d} ✅ Eklendi: \"{files[k_sel]}\" w={w_sel:.3f} → MAP@3={step_best_score:.5f}")
    
    # Belleği düzenli temizle
    if step % 10 == 0:  # Her 10 adımda bir temizle
        gc.collect()
        cp._default_memory_pool.free_all_blocks()

print(f"\n🎯 Final MAP@3 Skoru: {old_best:.6f}")
print(f"📊 Kullanılan model sayısı: {len(models)}")

# --- Final Ağırlık Vektörü ---
final_weights = np.zeros(x_train.shape[2], dtype=float)
for m, w in zip(models, weights):
    final_weights[m] = w

print(f"⚖️ Ağırlıklar toplamı: {final_weights.sum():.6f}")

# Seçilen modelleri göster
print("\n📋 Seçilen Modeller:")
for i, (m, w) in enumerate(zip(models, weights)):
    print(f"  {i+1:2d}. {files[m]:<30} = {w:6.3f}")

# --- Test Setine Uygula ---
print("\n🔮 Test tahminleri oluşturuluyor...")
x_test_gpu    = cp.array(x_test)
final_preds_gpu = cp.tensordot(x_test_gpu, cp.array(final_weights), axes=([2],[0]))
final_preds     = final_preds_gpu.get()

# Son temizlik
gc.collect()
cp._default_memory_pool.free_all_blocks()

print("✅ Tamamlandı! final_preds ile submission dosyanızı oluşturabilirsiniz.")

# Örnek submission:
# submission_df[classes] = final_preds
# submission_df.to_csv("submission.csv", index=False)

🚀 Hill-Climbing Ensemble Başlıyor (CV'siz)...
🔍 En iyi başlangıç modeli aranıyor...
0️⃣ Başlangıç modeli "top4_3_oof.npy" MAP@3 = 0.37696

⛰️ Hill-Climbing başlıyor...
 1 ✅ Eklendi: "xgb_1.4m_train_oof_trial_12.npy" w=0.430 → MAP@3=0.38017
 2 ✅ Eklendi: "xgb_1.4m_train_oof_trial_5.npy" w=0.150 → MAP@3=0.38113
 3 ✅ Eklendi: "top4_4_oof.npy" w=0.220 → MAP@3=0.38158
 4 ✅ Eklendi: "xgb_1.4m_train_oof_trial_4.npy" w=0.030 → MAP@3=0.38168
 5 ✅ Eklendi: "lgbm_goss_oof_trial_3.npy" w=0.010 → MAP@3=0.38172
⏹️ İyileşme -0.000016 < TOL (1e-05) — hill-climb sonlandı

🎯 Final MAP@3 Skoru: 0.381716
📊 Kullanılan model sayısı: 6
⚖️ Ağırlıklar toplamı: 1.840000

📋 Seçilen Modeller:
   1. top4_3_oof.npy                 =  1.000
   2. xgb_1.4m_train_oof_trial_12.npy =  0.430
   3. xgb_1.4m_train_oof_trial_5.npy =  0.150
   4. top4_4_oof.npy                 =  0.220
   5. xgb_1.4m_train_oof_trial_4.npy =  0.030
   6. lgbm_goss_oof_trial_3.npy      =  0.010

🔮 Test tahminleri oluşturuluyor...
✅ Tamamlandı!

In [18]:
import os
os.makedirs("submission_predictions", exist_ok=True)

np.save(f"submission_predictions/lb_38021_hc.npy", final_preds)


In [13]:
top3_indices = np.argsort(-final_preds, axis=1)[:, :3]
top3_labels  = np.vectorize(inverse_fert_dict.get)(top3_indices)


In [14]:
test = pd.read_csv('/kaggle/input/playground-series-s5e6/test.csv')

In [15]:
test_ids = test["id"].values

In [16]:
submission_df = pd.DataFrame({
    "id": test_ids,  # test setindeki satır ID’leri
    "Fertilizer Name": [' '.join(row) for row in top3_labels]
})
submission_df.to_csv("submission.csv", index=False)

In [17]:
s

NameError: name 's' is not defined